# Summary 
Generates the geojson hexes+dissolved cities file.

Demonstrates calculation of 50/10 access levels from the NBD map data against arbitrary 
geometry other than the precomputed hexagons.

Calculating this requires joining the geometry from the Canada wide hexagons geometry 
with the tabular `PHH_Speeds_Current-PHH_Vitesses_Actuelles.csv` dataset. 

Loading the data and executing the spatial joins takes a total of 7 minutes.

In [ ]:
import sys
sys.path.append("..")

%load_ext autoreload
%autoreload 1
%aimport src.datasets.joins
%aimport src.datasets.loading.statcan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from src.datasets.loading import statcan
from src.datasets.loading import ookla
from src.datasets import overlays

from src.datasets import joins
import src

In [ ]:
popctrs = statcan.boundary('population_centres')

In [ ]:
popctrs.loc[lambda s:s.PCNAME == "Lloydminster"]

In [ ]:
custom_areas = joins.hexagons_small_popctrs_combined()
o = custom_areas

In [ ]:
#phh_speed_csv = src.config.DATA_DIRECTORY / 'PHH' / 'CRTC_Speed_Data' / 'PHH_Speeds_Current-PHH_Vitesses_Actuelles.csv'
#phh_speeds = pd.read_csv(phh_speed_csv)
#phh_speeds = statcan.phh_csv_data()
#phh_geo = statcan.phh_geometry()
#phh = phh_geo.merge(phh_speeds, on='PHH_ID')
#phh = phh.merge(statcan.phh_data(), on='PHH_ID')
#phh = phh.loc[lambda s:s.Type !=8] # drop PHH nulls

phh = statcan.phh_geometry().merge(statcan.phh_data(), on='PHH_ID')
phh = phh.loc[lambda s:s.Type !=8] # drop PHH nulls

In [ ]:
o = joins.add_phh_pop(custom_areas, phh, 'HEXUID_PCPUID')

In [ ]:
del o['HEXuid_HEXidu'] # we don't want these, they're quite long in large cities

In [ ]:
popctrs = statcan.boundary('population_centres') # use this to generate a filtering region around cities

In [ ]:
#Can't plot the whole country, too many hexagons.
xmin, ymin, xmax, ymax = popctrs.loc[lambda s:s.PCNAME=="Lloydminster"].buffer(100000).total_bounds
o.cx[xmin:xmax,ymin:ymax].loc[lambda s:(s['Pop2016'] > 0) | (s['TDwell2016_TLog2016'] > 0) | (s['URDwell2016_RH2016']>0)].explore('Pop_Avail_50_10',  scheme='equalinterval', k = 4,)

In [ ]:
zeroable_cols = [
    'Pop2016', 'TDwell2016_TLog2016', 'URDwell2016_RH2016',
    'Pop2016_at_50_10_Combined', 'TDwell2016_at_50_10_Combined',
    'URDwell_at_50_10_Combined'
       ]
o[zeroable_cols] = o[zeroable_cols].fillna(0)

In [ ]:
o

In [ ]:
from pathlib import Path
from src import config

In [ ]:
derived_geometry = (Path(src.config.DATA_DIRECTORY) / 'processed' / 'geometries').resolve()
derived_geometry.mkdir(exist_ok=True, parents=True)

o.to_crs("EPSG:4326").to_file(derived_geometry / 'hexagons_w_dissolved_smaller_popctrs.geojson', driver='GeoJSON')

In [ ]:
o.to_file(derived_geometry / 'hexagons_w_dissolved_smaller_popctrs', driver="MapInfo File")